Code Libraries to Download

In [7]:
import requests
from time import sleep
from bs4 import BeautifulSoup as soup
from fpdf import FPDF
import unicodedata
import os

# Book Making


In [8]:
def clean_text(text):
    cleaned = ''.join(
        c for c in text
        if not unicodedata.category(c).startswith('C') and ord(c) <= 0xFFFF
    )
    return cleaned

In [10]:
# Config
book_name = 'History\'s Strongest Senior Brother'
chapter_count = 1836
book_limit = 15
font_path = 'Fonts/DejaVuSans.ttf'  # Make sure this exists

# Validate font path
if not os.path.exists(font_path):
    raise FileNotFoundError(f"Font not found at: {font_path}")

In [13]:
# PDF Setup
def new_pdf():
    pdf = FPDF()
    pdf.set_margins(left=10, top=20, right=10)
    pdf.add_font('DejaVu', '', font_path)
    pdf.set_font('DejaVu', '', 12)
    pdf.add_page()
    return pdf

In [14]:
pdf = new_pdf()
chapter_written = False  # Flag to check if anything was written

for i in range(1, chapter_count + 1):
    sleep(1)
    try:
        url = f'https://freewebnovel.com/novel/historys-strongest-senior-brother/chapter-{i}'
        html_page = requests.get(url).content
        soup_page = soup(html_page, "html.parser")

        # Get chapter name
        title = soup_page.title
        chapter_name = title.string.split(' | ')[0].split(' - ')[1].strip()

        content_div = soup_page.find('div', {'id': 'article'})
        if not content_div:
            print(f"[Warning] No content in chapter {i}. Skipping.")
            continue
        paragraphs = content_div.find_all("p")
        if not paragraphs:
            print(f"[Warning] No paragraphs in chapter {i}. Skipping.")
            continue

        if i % book_limit == 1:
            pdf.set_font("DejaVu", '', 28)
            pdf.multi_cell(0, 10, text=book_name, align='C')
            pdf.ln(10)

        pdf.set_font("DejaVu", '', 18)
        pdf.multi_cell(0, 10, text=clean_text(chapter_name))
        pdf.ln(5)

        pdf.set_font("DejaVu", '', 12)
        for j, p in enumerate(paragraphs):
            if j == 0:
                continue
            text = p.text.strip()
            if not text:
                continue
            if any(ord(c) > 0xFFFF for c in text):
                print(f"[Info] High Unicode in chapter {i}, paragraph {j}")
            text = clean_text(text)
            try:
                pdf.multi_cell(0, 8, txt=text)
                chapter_written = True
            except Exception as e:
                print(f"[Error] Writing paragraph {j} in chapter {i}: {e}")

        pdf.ln(10)

        if (i % book_limit == 0 or i == chapter_count) and chapter_written:
            start = i - book_limit + 1 if i >= book_limit else 1
            filename = f"History's Strongest Senior Brother/{book_name} Chapters {start} - {i}.pdf"
            print(f"[Output] Writing {filename}")
            pdf.output(filename)
            pdf = new_pdf()
            chapter_written = False

    except Exception as e:
        print(f"[Error] Failed at chapter {i}: {e}")

/var/folders/q6/gd7hnxg17r3dg8bfxfkd3h_00000gn/T/ipykernel_44764/136127658.py:44: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.multi_cell(0, 8, txt=text)


[Error] Writing paragraph 2 in chapter 1: Not enough horizontal space to render a single character
[Error] Writing paragraph 3 in chapter 1: Not enough horizontal space to render a single character
[Error] Writing paragraph 4 in chapter 1: Not enough horizontal space to render a single character
[Error] Writing paragraph 5 in chapter 1: Not enough horizontal space to render a single character
[Error] Writing paragraph 6 in chapter 1: Not enough horizontal space to render a single character
[Error] Writing paragraph 7 in chapter 1: Not enough horizontal space to render a single character
[Error] Writing paragraph 8 in chapter 1: Not enough horizontal space to render a single character
[Error] Writing paragraph 9 in chapter 1: Not enough horizontal space to render a single character
[Error] Writing paragraph 10 in chapter 1: Not enough horizontal space to render a single character
[Error] Writing paragraph 11 in chapter 1: Not enough horizontal space to render a single character
[Error] 

Font MPDFAA+DejaVuSansBook is missing the following glyphs: '〃' (\u3003), '￣' (\uffe3), 'ゞ' (\u309e)


[Error] Writing paragraph 2 in chapter 15: Not enough horizontal space to render a single character
[Error] Writing paragraph 3 in chapter 15: Not enough horizontal space to render a single character
[Error] Writing paragraph 4 in chapter 15: Not enough horizontal space to render a single character
[Error] Writing paragraph 5 in chapter 15: Not enough horizontal space to render a single character
[Error] Writing paragraph 6 in chapter 15: Not enough horizontal space to render a single character
[Error] Writing paragraph 7 in chapter 15: Not enough horizontal space to render a single character
[Error] Writing paragraph 8 in chapter 15: Not enough horizontal space to render a single character
[Error] Writing paragraph 9 in chapter 15: Not enough horizontal space to render a single character
[Error] Writing paragraph 10 in chapter 15: Not enough horizontal space to render a single character
[Error] Writing paragraph 11 in chapter 15: Not enough horizontal space to render a single characte

KeyboardInterrupt: 